# Completion Trie and metrics

In [1]:
from pyquickhelper.ipythonhelper import add_notebook_menu
add_notebook_menu()

## Wikipedia titles, uniform

In [2]:
from mlstatpy.data.wikipedia import download_titles
file_titles = download_titles(country='fr')

In [3]:
from mlstatpy.data.wikipedia import enumerate_titles
list_titles = list(sorted(_ for _ in enumerate_titles(file_titles) if 'A' <= _[0] <= 'Z'))

In [4]:
len(list_titles), list_titles[:5], list_titles[1000000:1000005]

(3045736,
 ['A',
  'A & A',
  'A (Airport Express)',
  'A (Ayumi Hamasaki)',
  "A (Disque d'Ayumi Hamasaki)"],
 ['Festival du cinéma de Brive - Rencontres du moyen métrage',
  'Festival du cinéma de Brive - Rencontres européennes du moyen métrage',
  'Festival du cinéma de Venise',
  'Festival du cinéma espagnol de Malaga',
  'Festival du cinéma européen de Lille'])

In [21]:
from mlstatpy.nlp.completion import CompletionTrieNode

def gain_dynamique_moyen_par_mot(queries, weights):
    per = list(zip(weights, queries))
    total = sum(weights) * 1.0 
    res = []
    trie = CompletionTrieNode.build([(None, q) for _, q in per])
    trie.precompute_stat()
    trie.update_stat_dynamic()
    wks = [(w, p, len(w)-trie.min_keystroke(w)[0]) for p, w in per]
    wks_dyn = [(w, p, len(w)-trie.min_dynamic_keystroke(w)[0]) for p, w in per]
    wks_dyn2 = [(w, p, len(w)-trie.min_dynamic_keystroke2(w)[0]) for p, w in per]
    gain = sum( g*p/total for w, p, g in wks)
    gain_dyn = sum( g*p/total for w, p, g in wks_dyn)
    gain_dyn2 = sum( g*p/total for w, p, g in wks_dyn2)
    ave_length = sum( len(w) * p / total for p, w in per)
    return gain, gain_dyn, gain_dyn2, ave_length

In [23]:
import time, random
print("time", 0)
allres = []
for i in range(1, 11):
    begin = time.clock()    
    spl = random.sample(list_titles, i * 100)
    spl.sort()
    res = gain_dynamique_moyen_par_mot(spl, [1.0] * len(spl))
    print("time: {0}s - nb={1}".format(time.clock() - begin, len(spl)), "gain", res)
    print("time: {0}s - nb={1}".format(time.clock() - begin, len(spl)), "gain", tuple(_/res[-1] for _ in res))
    allres = res

time 0
time: 0.7003269698371923s - nb=100 gain (16.190000000000005, 16.190000000000005, 16.190000000000005, 19.319999999999993)
time: 0.7004954580870617s - nb=100 gain (0.8379917184265016, 0.8379917184265016, 0.8379917184265016, 1.0)
time: 2.0230538110416774s - nb=200 gain (15.434999999999997, 15.434999999999997, 15.434999999999997, 18.825)
time: 2.023238549427333s - nb=200 gain (0.8199203187250995, 0.8199203187250995, 0.8199203187250995, 1.0)
time: 4.6613253610736365s - nb=300 gain (16.55666666666665, 16.55666666666665, 16.55666666666665, 20.186666666666667)
time: 4.661508388918719s - nb=300 gain (0.8201783355350057, 0.8201783355350057, 0.8201783355350057, 1.0)
time: 6.34021327020298s - nb=400 gain (15.182500000000005, 15.182500000000005, 15.182500000000005, 18.947500000000012)
time: 6.340411265278362s - nb=400 gain (0.8012930465760651, 0.8012930465760651, 0.8012930465760651, 1.0)
time: 10.30483843516322s - nb=500 gain (16.35999999999998, 16.36199999999998, 16.36199999999998, 20.37199